<a href="https://colab.research.google.com/github/Vinusha06/HackerRank-Solution/blob/master/Mini_Hackathon_Data_Munging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Objective

At the end of this experiment, you will be able to:

* Perform Data preprocessing

## Dataset

### Description

We will be using district wise demographics, enrollments, school and teacher indicator data to predict whether the literacy rate is high / medium / low in each district.

### Data Preprocessing

Data preprocessing is an important step of solving every machine learning problem. Most of
the datasets used with Machine Learning problems need to be processed / cleaned / transformed
so that a Machine Learning algorithm can be trained on it.

There are different steps involved for Data Preprocessing. These steps are as follows:

    1. Data Cleaning → In this step the primary focus is on
        -Handling missing data
        -Handling nosiy data
        -Detection and removal of outliers
    
    2. Data Integration → This process is used when data is gathered from various data sources
    and data are combined to form consistent data. This data after performing cleaning is used
    for analysis.
    
    3. Data Transformation → In this step we will convert the raw data into a specified for-
    mat according to the need of the model we are building. There are many options used for
    transforming the data as below:
        -Normalization
        -Aggregation
        -Generalization
        
    4. Data Reduction → After data transformation and scaling the redundancy within the data
    is removed and efficiently organizing the data is performed.



In [ ]:
# Run this cell to download the data (you will get the zip file)
!wget https://cdn.talentsprint.com/aiml/Experiment_related_data/data-20190108T113429Z-001.zip


In [ ]:

# Run this cell to unzip the data
!unzip data-20190108T113429Z-001.zip

In [ ]:
!ls

In [ ]:


%cd data

In [ ]:
!ls

#### Exercise 1 
We have four different files

* Districtwise_Basicdata.csv
* Districtwise_Enrollment_details_indicator.csv
* Districtwise_SchoolData.csv
* Districtwise_Teacher_indicator.csv
These files contain the neccesary data to solve the problem.
Load all the files correctly, after observing the header level details, data records etc

Hint : Use read_csv from pandas

In [ ]:
import pandas as pd


In [ ]:

# Your Code Here


basicdata = pd.read_csv('Districtwise_Basicdata.csv')


basicdata.head()

In [ ]:
enrolldata = pd.read_csv('Districtwise_Enrollment_details_indicator.csv',skiprows=2)
enrolldata.head()

In [ ]:

schooldata = pd.read_csv('Districtwise_SchoolData.csv',skiprows=2)

schooldata.head()

In [ ]:
teacherdata = pd.read_csv('Districtwise_Teacher_indicator.csv',skiprows=2)

teacherdata.head()

#### Exercise 2  

* Remove the unwanted columns, which are unlikely to contribute for the prediction of overall literacy grade. The decision of what constitutes unwanted columns depends on how it effects your final accuracy (and very little on your domain understanding of education sector in India; you're encouraged however to exercise some domain understanding too if you wish)

**Hint** use pandas drop function to drop your choice of unwanted columns (if any).


* As the required data is present in different files, we need to integrate all the four to make single dataframe/dataset. For that purpose, create a unique identifier for each row in all the dataframes so that it can be used to map the data in different files correctly
* Join/integrate this data 

Example : data of the district ananthapur in Andrapradesh, which present in different files should form a single row 

Hint : 
* Use the combination of year, statecode, district code as unique identifier 

* Refer the following link for merge, join and concat syntaxes:  

https://pandas.pydata.org/pandas-docs/stable/merging.html


In [ ]:
# Your Code Here
basicdata = basicdata.drop(['0-6 Population','Number of Blocks','Number of Villages','Number of Clusters','Sex Ratio 0-6'],axis=1)
basicdata

In [ ]:

enrolldata = enrolldata.drop(['No Response'],axis = 1)
enrolldata = enrolldata.rename({'Distcd':'District Code',"Distname":'District Name'},axis=1)
enrolldata.head()

In [ ]:
schooldata = schooldata.drop(['No Response','Academic Year'],axis = 1)
schooldata.head()


In [ ]:
teacherdata = teacherdata.drop(['No Response','Academic Year'],axis = 1)
teacherdata.head()

In [ ]:


df = [basicdata,enrolldata,schooldata,teacherdata]
result = pd.concat(df,axis = 1,join='inner')


result = result.loc[:,~result.columns.duplicated()]
result.head()

In [ ]:
result

Follow this steps in order to clean the data:

#### Exercise 3 

* Overall_lit is our target variable, which we need to predict. Delete the row with missing overall_lit column
* Take a call to replace the missing values in any other column appropriately with mean/median/mode
* Convert categorical values to numerical values
Example : If a feature contains categorical values such as dog, cat, mouse etc then replace them with 1, 2, 3 etc or using one hot encoding (your judgement)

*Hint* :
* Use pandas fillna function to replace the missing values

In [ ]:
# Your Code Here
result = result.dropna(subset = ['Overall Literacy'])


In [ ]:

result

In [ ]:

result['Overall Literacy'].isna().sum()

In [ ]:

import numpy as np


In [ ]:

result = result.fillna(result.mean())
#result1 =result.replace(np.nan,result.mean(axis =1))

In [ ]:
result.isna().sum().any()

In [ ]:
result = result.replace({'High':3,'Medium':2,'Low':1})

In [ ]:
result = result.dropna()

#### Exercise 4 

Use the functions below to adjust the outliers

smooth_out function takes pandas dataframe as input and caculates mean, standard deviation of every column to check whether all the values in that lies within the range of mean +/- 2*standard_deviation of that column or not.
If any of the values are not present in that boundary, then that values is brought on to the boundary.

**Hint:** Should  the index column be normalized too? 

<img src="https://cdn.talentsprint.com/aiml/Experiment_related_data/normal_dist.png">

In [ ]:
# Function to clip and clam the data
def clip_clamp(x, mean, sd):
    # Checking whether the value is less than a differenced value between mean and standard deviation.
    if x < mean - 2*sd :
        return mean - 2*sd
    #Checking whether the value is greater than a differenced value between mean and standard deviation.
    elif x > mean + 2*sd :
        return mean + 2*sd
    # If above two conditions are not statisfied we will return the original value
    else :
        return x

In [ ]:
# Function to smooth the data
def smooth_out(Total_data):
    for i in Total_data.columns:
        # Calculating the mean value
        mean = np.mean(Total_data[i].values, axis=0)
        # Calculating the standard deviation value
        sd = np.std(Total_data[i].values, axis=0)
        # Calculating the corrected value using clip and clamp function
        corrected = np.array([clip_clamp(x, mean, sd) for x in Total_data[i].values])
        # Storing the data in form of series
        Total_data[i] = pd.Series(corrected, index=Total_data[i].index)
    return Total_data

In [ ]:

# Your Code Here

datastore = result.iloc[:,[0,1,2,3,4]]
data = result.drop(result.iloc[:,[0,1,2,3,4]],axis = 1)

In [ ]:
data = data.drop([0])

In [ ]:

data = data.astype(float)


In [ ]:

data = smooth_out(data)


In [ ]:
data

#### Exercise 5 

Use the function below (corr_features) to identify uncorrelated features and remove the remaining features
* corr_features takes pandas dataframe, columns in the dataframe and bar (corelation co-efficient)

In [ ]:
# Function to find uncorrelated features
def corr_features(df,cols,bar=0.9):
    for c,i in enumerate(cols[:-1]):
        col_set = set(cols)
        for j in cols[c+1:]:
            if i==j:
                continue
           
            score = df[i].corr(df[j])
            
            if score>bar:
                cols = list(col_set-set([j]))
            if score<-bar:
                cols = list(col_set-set([j]))
    return cols

In [ ]:
# Your Code Here
uncorrelated = corr_features(data,data.iloc[:,:])

In [ ]:
results = data[uncorrelated]

In [ ]:

results

#### Exercise 6 

Perform Mean Correction and Standard Scaling on the data feature/column wise.

**Hint:** In order to understand the idea behind the terms used above, you may refer the following link: 

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Your Code Here
from sklearn.preprocessing import StandardScaler

In [ ]:

scaler = StandardScaler()

In [ ]:
print(scaler.fit(results))

In [ ]:
print(scaler.mean_)

In [ ]:
print(scaler.transform(results))

#### Exercise 7 **(Optional)**

you can apply different classifiers(from sklearn) on the preprocessed data .

In [ ]:
def callKnn(data,targets):
    X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=0.33)
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train, y_train)
    predicted_labels = neigh.predict(X_test)
    return accuracy_score(y_test,predicted_labels)

In [ ]:
# Your Code Here
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
callKnn(results,data['Overall Literacy'])